In [ ]:
# Run the base analysis from Part_2A_bonus.ipynb
%run "Part_2A_bonus.ipynb"

## Question 2.a Part (i): Dual Formulation and Economic Interpretation

### Task i: Mathematical Formulation

Building on the optimization model from Question 1.B, we consider a prosumer with photovoltaic (PV) generation and flexible demand. The consumer aims to minimize total operational costs while respecting comfort preferences.

#### Input Data

The problem uses the following parameters:

In [ ]:
# Display key system parameters
print("System Parameters:")
print("="*60)
print(f"Planning horizon (T): {T} hours")
print(f"Import tariff (τ_imp): {tau_imp} DKK/kWh")
print(f"Export tariff (τ_exp): {tau_exp} DKK/kWh")
print(f"Maximum import (P_imp_max): {max_import} kW")
print(f"Maximum export (P_exp_max): {max_export} kW")
print(f"PV capacity: {pv_max_power} kW")
print(f"Maximum demand (D_max): {D_max} kW")
print(f"Discomfort penalty (α): {alpha} DKK/kWh")
print(f"\nElectricity price range: {min(electricity_prices):.3f} - {max(electricity_prices):.3f} DKK/kWh")
print(f"Total reference load: {sum(D_ref):.2f} kWh")

#### Objective Function

The consumer minimizes total operational cost:

$$\min \sum_{t=1}^{T} \left[ P_{imp,t} \cdot (\tau_{imp} + \lambda_t) - P_{exp,t} \cdot (\lambda_t - \tau_{exp}) \right] + \alpha \sum_{t=1}^{T} L_t$$

Where:
- First term: Grid import costs
- Second term: Grid export revenue (subtracted from cost)
- Third term: Discomfort penalty for deviating from preferred consumption

#### Decision Variables

- $D_t \geq 0$: Demand consumption at hour $t$ [kWh]
- $C_t \geq 0$: PV curtailment at hour $t$ [kWh]
- $P_{imp,t} \geq 0$: Grid import at hour $t$ [kWh]
- $P_{exp,t} \geq 0$: Grid export at hour $t$ [kWh]
- $L_t \geq 0$: Discomfort (absolute load deviation) at hour $t$ [kWh]

#### Constraints

1. **Power balance** (∀t ∈ T):
   $$P_{imp,t} - P_{exp,t} = D_t - P^{PV}_t + C_t$$

2. **PV curtailment limit** (∀t ∈ T):
   $$C_t \leq P^{PV}_t$$

3. **Discomfort upper bound** (∀t ∈ T):
   $$L_t \geq D_t - D_{ref,t}$$

4. **Discomfort lower bound** (∀t ∈ T):
   $$L_t \geq -(D_t - D_{ref,t})$$

5. **Variable bounds**:
   $$0 \leq D_t \leq D_{max}, \quad 0 \leq P_{imp,t} \leq P_{imp,max}, \quad 0 \leq P_{exp,t} \leq P_{exp,max}$$

**Important:** There is **no daily energy requirement constraint**. Deviations from the reference load $D_{ref,t}$ are penalized through the discomfort cost $\alpha \sum_t L_t$, providing complete temporal flexibility while accounting for comfort preferences.

### Task ii: Dual Problem, Lagrangian and KKT

#### The Lagrangian

We introduce Lagrange multipliers for each constraint:

- $\pi_t \in \mathbb{R}$: Power balance constraint (equality)
- $\mu_t \geq 0$: PV curtailment limit
- $\omega_{1,t} \geq 0$: Discomfort upper bound
- $\omega_{2,t} \geq 0$: Discomfort lower bound

The Lagrangian function combines the objective with all constraints:

$$
\begin{align}
\mathcal{L} &= \sum_{t=1}^{T} \left[ P_{imp,t}(\tau_{imp} + \lambda_t) - P_{exp,t}(\lambda_t - \tau_{exp}) + \alpha L_t \right] \\
&+ \sum_{t=1}^{T} \pi_t \left( P_{imp,t} - P_{exp,t} - D_t + P^{PV}_t - C_t \right) \\
&+ \sum_{t=1}^{T} \mu_t \left( C_t - P^{PV}_t \right) \\
&+ \sum_{t=1}^{T} \omega_{1,t} \left( D_t - D_{ref,t} - L_t \right) \\
&+ \sum_{t=1}^{T} \omega_{2,t} \left( -D_t + D_{ref,t} - L_t \right) \\
&+ \text{(variable bound terms)}
\end{align}
$$

#### KKT Stationarity Conditions

At optimality, the gradient of $\mathcal{L}$ with respect to each primal variable equals zero:

$$\frac{\partial \mathcal{L}}{\partial P_{imp,t}} = (\tau_{imp} + \lambda_t) + \pi_t = 0 \quad \Rightarrow \quad \pi_t = -(\tau_{imp} + \lambda_t)$$

$$\frac{\partial \mathcal{L}}{\partial P_{exp,t}} = -(\lambda_t - \tau_{exp}) - \pi_t = 0 \quad \Rightarrow \quad \pi_t = -(\lambda_t - \tau_{exp})$$

$$\frac{\partial \mathcal{L}}{\partial D_t} = -\pi_t + \omega_{1,t} - \omega_{2,t} = 0$$

$$\frac{\partial \mathcal{L}}{\partial C_t} = -\pi_t + \mu_t = 0 \quad \Rightarrow \quad \mu_t = \pi_t$$

$$\frac{\partial \mathcal{L}}{\partial L_t} = \alpha - \omega_{1,t} - \omega_{2,t} = 0 \quad \Rightarrow \quad \omega_{1,t} + \omega_{2,t} = \alpha$$

These conditions reveal the economic interpretation of dual variables.

### Qualitative Analysis

#### Economic Interpretation of Dual Variables

**1. Power Balance Dual ($\pi_t$)**

The dual variable $\pi_t$ represents the **marginal value of energy** at hour $t$:

- **When importing:** $\pi_t = -(\tau_{imp} + \lambda_t)$ → marginal cost equals full import price
- **When exporting:** $\pi_t = -(\lambda_t - \tau_{exp})$ → marginal value equals net export revenue
- **When self-sufficient:** $\pi_t$ reflects internal energy valuation

The negative sign arises from our Lagrangian formulation. The absolute value $|\pi_t|$ represents the **shadow price of energy** - how much the objective would improve if 1 additional kWh became available at hour $t$.

**2. PV Curtailment Dual ($\mu_t$)**

From KKT: $\mu_t = \pi_t$. This dual variable indicates:

- $\mu_t > 0$: PV is being curtailed; additional PV utilization capacity would be valuable
- $\mu_t = 0$: No curtailment; all available PV is used

Positive $\mu_t$ signals potential value in battery storage or increased export capacity.

**3. Discomfort Duals ($\omega_{1,t}$, $\omega_{2,t}$)**

These duals quantify the marginal cost of deviating from preferred consumption:

- $\omega_{1,t} > 0$: Load exceeds reference ($D_t > D_{ref,t}$)
- $\omega_{2,t} > 0$: Load below reference ($D_t < D_{ref,t}$)
- $\omega_{1,t} + \omega_{2,t} = \alpha$: Constraint from KKT stationarity

These duals directly represent the penalty for comfort deviation, balancing cost savings against user preferences.

### Model Implementation

The model was implemented using Gurobi 12.0.3 optimizer in Python. After solving the primal problem, dual variables are extracted from constraint shadow prices:

In [ ]:
# Display optimal solution summary
print("\n" + "="*70)
print("OPTIMAL SOLUTION SUMMARY")
print("="*70)

if model.status == GRB.OPTIMAL:
    print(f"\nObjective Value (Total Cost): {model.objVal:.2f} DKK")
    
    # Cost breakdown
    import_cost = sum(P_imp_t[t].X * (tau_imp + electricity_prices[t]) for t in Times)
    export_revenue = sum(P_exp_t[t].X * (electricity_prices[t] - tau_exp) for t in Times)
    discomfort_cost = alpha * sum(L_t[t].X for t in Times)
    
    print(f"\nCost Breakdown:")
    print(f"  Import cost:        {import_cost:8.2f} DKK")
    print(f"  Export revenue:    -{export_revenue:8.2f} DKK")
    print(f"  Net energy cost:    {import_cost - export_revenue:8.2f} DKK")
    print(f"  Discomfort cost:    {discomfort_cost:8.2f} DKK")
    print(f"  " + "-"*40)
    print(f"  Total cost:         {model.objVal:8.2f} DKK")
    
    # Energy summary
    total_import = sum(P_imp_t[t].X for t in Times)
    total_export = sum(P_exp_t[t].X for t in Times)
    total_load = sum(D_t[t].X for t in Times)
    total_ref = sum(D_ref)
    total_pv = sum(pv_prod_hourly)
    total_curtailment = sum(C_t[t].X for t in Times)
    
    print(f"\nEnergy Summary:")
    print(f"  Total PV production:  {total_pv:.2f} kWh")
    print(f"  Total PV curtailment: {total_curtailment:.2f} kWh ({100*total_curtailment/total_pv:.1f}%)")
    print(f"  Total grid import:    {total_import:.2f} kWh")
    print(f"  Total grid export:    {total_export:.2f} kWh")
    print(f"  Total load:           {total_load:.2f} kWh")
    print(f"  Reference load:       {total_ref:.2f} kWh")
    print(f"  Load deviation:       {total_load - total_ref:+.2f} kWh ({100*(total_load - total_ref)/total_ref:+.1f}%)")
    
    # Strong duality verification
    print(f"\nStrong Duality Check:")
    print(f"  Primal objective: {model.objVal:.6f} DKK")
    print(f"  Dual bound:       {model.ObjBound:.6f} DKK")
    print(f"  Gap:              {abs(model.objVal - model.ObjBound):.10f} DKK (verified)")

### Numerical Analysis

#### Dual Variable Statistics

In [ ]:
# Detailed dual variable analysis
print("\n" + "="*70)
print("DUAL VARIABLE ANALYSIS")
print("="*70)

print(f"\n1. Energy Shadow Price (π_t):")
print(f"   Mean:    {np.mean(np.abs(mu_t)):.4f} DKK/kWh")
print(f"   Min:     {np.min(np.abs(mu_t)):.4f} DKK/kWh (Hour {np.argmin(np.abs(mu_t))})")
print(f"   Max:     {np.max(np.abs(mu_t)):.4f} DKK/kWh (Hour {np.argmax(np.abs(mu_t))})")
print(f"   Std Dev: {np.std(np.abs(mu_t)):.4f} DKK/kWh")
print(f"\n   Interpretation: Shadow price represents marginal energy value.")
print(f"   High values indicate hours where energy is most valuable.")

print(f"\n2. PV Curtailment Duals (μ_t):")
curtailment_hours = sum(1 for v in nu_t if abs(v) > 1e-6)
print(f"   Hours with curtailment: {curtailment_hours} out of {T}")
if curtailment_hours > 0:
    curtailed_hours = [t for t in Times if abs(nu_t[t]) > 1e-6]
    print(f"   Curtailment occurs at hours: {curtailed_hours}")
    print(f"   Mean dual (when curtailing): {np.mean([abs(nu_t[t]) for t in curtailed_hours]):.4f} DKK/kWh")
    print(f"\n   Interpretation: Positive duals indicate PV is curtailed.")
    print(f"   Signals potential value in storage or export capacity.")
else:
    print(f"\n   Interpretation: No PV curtailment - all production utilized.")

print(f"\n3. Discomfort Constraint Duals (ω1_t, ω2_t):")
hours_above_ref = sum(1 for v in omega1_t if abs(v) > 1e-6)
hours_below_ref = sum(1 for v in omega2_t if abs(v) > 1e-6)
hours_at_ref = T - hours_above_ref - hours_below_ref
print(f"   Hours with load > reference: {hours_above_ref}")
print(f"   Hours with load < reference: {hours_below_ref}")
print(f"   Hours at reference:          {hours_at_ref}")
print(f"\n   Interpretation: Most hours show load deviation.")
print(f"   Model balances cost savings against comfort penalties.")

# Verify KKT condition: ω1_t + ω2_t = α
print(f"\n4. KKT Condition Verification (ω1_t + ω2_t = α):")
for t in [0, 6, 12, 18]:  # Sample hours
    sum_omega = omega1_t[t] + omega2_t[t]
    error = abs(sum_omega - alpha)
    status = "✓" if error < 1e-6 else "✗"
    print(f"   Hour {t:2d}: ω1 + ω2 = {sum_omega:.6f}, α = {alpha:.6f}, error = {error:.8f} {status}")

#### Visualization of Dual Variables

The energy shadow price $\pi_t$ (represented by $\mu_t$ in Gurobi output) reveals the temporal variation in energy value:

In [ ]:
# Display the shadow price visualization (already created in Part_2A_bonus.ipynb)
from IPython.display import Image, display
import os

if os.path.exists('../Plots/Scenarios 2)/part_2a_i_shadow_prices.png'):
    display(Image('../Plots/Scenarios 2)/part_2a_i_shadow_prices.png'))
else:
    print("Shadow price plot not found. Please run Part_2A_bonus.ipynb first.")

The upper plot shows how the energy shadow price $|\pi_t|$ compares with import/export price bounds:

- **Red dashed line:** $\lambda_t + \tau_{imp}$ (full cost of importing)
- **Green dashed line:** $\lambda_t - \tau_{exp}$ (net value of exporting)
- **Gray zone:** "No-trade zone" where self-consumption is optimal

The lower plot shows the resulting import/export decisions, confirming:
- Import when $|\pi_t| = \tau_{imp} + \lambda_t$ (high energy value)
- Export when $|\pi_t| = \lambda_t - \tau_{exp}$ (low energy value, excess PV)

#### Key Findings

1. **Strong Duality Holds:** The primal and dual objectives are equal (within numerical precision), confirming optimality.

2. **Energy Value Varies Significantly:** The range of shadow prices demonstrates substantial temporal variation in energy value, quantifying the benefit of flexibility.

3. **Tariff Impact:** Import/export tariffs create a "no-trade zone" reducing market participation at intermediate prices.

4. **No Hard Daily Constraint:** Complete flexibility is maintained - consumers can deviate from reference load with only penalty costs, not hard constraints.

5. **Investment Signals:** PV curtailment duals indicate potential value in storage or export capacity expansion.

---

## Question 2.a Part (ii): Hourly Demand and Supply Curves

### Methodology

To derive the consumer's demand (import) and supply (export) curves as functions of electricity price:

1. Select representative hours for analysis
2. Vary the electricity price $\lambda_t$ at the selected hour
3. Resolve the optimization for each price level
4. Record optimal $P_{imp,t}$ and $P_{exp,t}$
5. Plot the resulting demand and supply curves

This reveals the consumer's **price-responsive behavior** and optimal **market participation strategy**.

### Economic Theory

From the KKT stationarity conditions:

**Marginal Willingness to Pay (WTP):**
- Consumer imports when: $\lambda_t \leq |\pi_t| - \tau_{imp}$
- Maximum price willing to pay for imports

**Marginal Opportunity Cost (MOC):**  
- Consumer exports when: $\lambda_t \geq |\pi_t| + \tau_{exp}$
- Minimum price required to export

**No-Trade Zone:**
- When $MOC \leq \lambda_t \leq WTP$: Self-consumption is optimal
- Width determined by internal energy value and tariff structure

### Implementation

We analyze three representative hours:

In [ ]:
# Display characteristics of selected hours
print("\n" + "="*70)
print("REPRESENTATIVE HOUR CHARACTERISTICS")
print("="*70)

for hour, label in zip(analysis_hours, hour_labels):
    print(f"\n{label} (Hour {hour}):")
    print(f"  Base electricity price: {electricity_prices[hour]:.3f} DKK/kWh")
    print(f"  PV production:          {pv_prod_hourly[hour]:.2f} kW")
    print(f"  Reference load:         {D_ref[hour]:.2f} kW")
    print(f"  Optimal load:           {D_t[hour].X:.2f} kW")
    print(f"  Grid import:            {P_imp_t[hour].X:.2f} kW")
    print(f"  Grid export:            {P_exp_t[hour].X:.2f} kW")
    print(f"  Shadow price |π_t|:     {abs(mu_t[hour]):.3f} DKK/kWh")

The demand/supply curves are derived by varying electricity price across a wide range:

In [ ]:
print(f"\nPrice sensitivity analysis:")
print(f"  Price range tested: {price_range[0]:.2f} to {price_range[-1]:.2f} DKK/kWh")
print(f"  Number of price points: {len(price_range)}")
print(f"\nCurves derived successfully for hours: {analysis_hours}")

### Numerical Analysis

#### Demand Curves (Import)

In [ ]:
# Display demand curve visualizations
if os.path.exists('../Plots/Scenarios 2)/part_2a_ii_demand_curves.png'):
    display(Image('../Plots/Scenarios 2)/part_2a_ii_demand_curves.png'))
else:
    print("Demand curves plot not found. Please run Part_2A_bonus.ipynb first.")

**Observations:**

- **Downward sloping:** Higher prices → reduced imports (expected demand curve behavior)
- **Time variation:** Curves differ by hour due to PV availability and load requirements
- **Price thresholds:** Each hour has a distinct WTP beyond which imports cease

#### Supply Curves (Export)

In [ ]:
# Display supply curve visualizations  
if os.path.exists('../Plots/Scenarios 2)/part_2a_ii_supply_curves.png'):
    display(Image('../Plots/Scenarios 2)/part_2a_ii_supply_curves.png'))
else:
    print("Supply curves plot not found. Please run Part_2A_bonus.ipynb first.")

**Observations:**

- **Upward sloping:** Higher prices → increased exports (expected supply curve behavior)
- **Midday advantage:** Hour 12 shows strong export capability due to PV production
- **Evening constraint:** Hour 18 has limited export (no PV available)

#### Combined Analysis

In [ ]:
# Display combined curves
if os.path.exists('../Plots/Scenarios 2)/part_2a_ii_combined_curves.png'):
    display(Image('../Plots/Scenarios 2)/part_2a_ii_combined_curves.png'))
else:
    print("Combined curves plot not found. Please run Part_2A_bonus.ipynb first.")

#### Price Threshold Analysis

In [ ]:
# Analyze and display price thresholds
print("\n" + "="*70)
print("PRICE THRESHOLD ANALYSIS")
print("="*70)

for hour, label in zip(analysis_hours, hour_labels):
    data = curves[hour]
    
    # Find thresholds
    import_data = data[data['import'] > 0.01]
    export_data = data[data['export'] > 0.01]
    
    print(f"\n{label} (Hour {hour}):")
    print(f"  Base case price: {electricity_prices[hour]:.3f} DKK/kWh")
    print(f"  Shadow price:    {abs(mu_t[hour]):.3f} DKK/kWh")
    
    if len(import_data) > 0:
        max_wtp = import_data['price'].max()
        print(f"\n  Import behavior:")
        print(f"    Max WTP:           {max_wtp:.3f} DKK/kWh")
        print(f"    Theoretical WTP:   {abs(mu_t[hour]) - tau_imp:.3f} DKK/kWh")
        print(f"    Import range:      {import_data['price'].min():.3f} - {max_wtp:.3f} DKK/kWh")
    
    if len(export_data) > 0:
        min_moc = export_data['price'].min()
        print(f"\n  Export behavior:")
        print(f"    Min MOC:           {min_moc:.3f} DKK/kWh")
        print(f"    Theoretical MOC:   {abs(mu_t[hour]) + tau_exp:.3f} DKK/kWh")
        print(f"    Export range:      {min_moc:.3f} - {export_data['price'].max():.3f} DKK/kWh")
    
    # Base case action
    base_row = data[data['price'].round(4) == round(electricity_prices[hour], 4)]
    if len(base_row) > 0:
        imp = base_row['import'].values[0]
        exp = base_row['export'].values[0]
        print(f"\n  Base case action:  ", end="")
        if imp > 0.01:
            print(f"IMPORTING {imp:.2f} kW")
        elif exp > 0.01:
            print(f"EXPORTING {exp:.2f} kW")
        else:
            print(f"SELF-SUFFICIENT (no grid trade)")

### Qualitative Discussion

#### Curve Characteristics

**Demand Curve (Import $P_{imp,t}$ vs. $\lambda_t$):**

- **Shape:** Downward sloping - economic rationality confirmed
- **At low prices:** Import maximizes (cheap electricity incentivizes consumption)
- **At high prices:** Import reduces to zero (expensive, prefer internal resources)
- **Slope:** Indicates price elasticity of demand

**Supply Curve (Export $P_{exp,t}$ vs. $\lambda_t$):**

- **Shape:** Upward sloping - higher prices incentivize export
- **At low prices:** No export (internal value exceeds market price)
- **At high prices:** Export maximizes (monetize excess PV)
- **Slope:** Indicates export responsiveness to price signals

#### Time-of-Day Variations

Curves differ significantly across hours due to:

1. **PV Availability:**
   - Midday (Hour 12): High PV → strong export potential, reduced import needs
   - Morning/Evening: Low/no PV → higher import propensity, limited export

2. **Load Requirements:**
   - Peak hours: Higher demand → more import propensity
   - Off-peak: Lower demand → more export potential

3. **Flexibility Constraints:**
   - Discomfort penalty ($\alpha$) limits load shifting
   - Affects price responsiveness uniformly across hours

4. **No Daily Constraint:**
   - Each hour optimized independently (subject to discomfort)
   - Maximum temporal flexibility achieved

#### Market Participation Strategy

Based on derived curves, optimal bidding strategy:

**For Imports (Demand Bids):**
- Submit downward-sloping bid curve to day-ahead market
- Quantity at each price determined by derived demand curve
- Import when market clears below WTP threshold

**For Exports (Supply Bids):**
- Submit upward-sloping bid curve to day-ahead market
- Quantity at each price determined by derived supply curve  
- Export when market clears above MOC threshold

**No-Trade Zone:**
- When $MOC \leq \lambda_t \leq WTP$: Self-consumption optimal
- Width = $2|\pi_t| + (\tau_{exp} + \tau_{imp})$
- Tariffs directly impact market participation range

#### Value of Flexibility

The derived curves quantify flexibility value:

1. **Price Elasticity:** Flatter curves → more flexible → higher value
2. **Arbitrage Potential:** Wide active trading range → profitable opportunities
3. **Market Revenue:** Area under supply curve above base price
4. **Cost Savings:** Area under demand curve below base price
5. **Temporal Arbitrage:** Different WTP/MOC across hours enables load shifting profits

### Key Findings

1. **Clear Price Responsiveness:** Consumer exhibits rational economic behavior with well-defined threshold prices derived from shadow prices.

2. **Time-Dependent Flexibility:** Demand/supply curves vary significantly by hour, reflecting PV production patterns and load requirements.

3. **Tariff Structure Impact:** Import/export tariffs create a no-trade zone, reducing market participation at intermediate prices.

4. **Actionable Bidding:** Curves provide direct input for day-ahead market bid submission, enabling optimal market participation.

5. **Quantified Flexibility Value:** The shape and position of curves reveal the economic benefit of demand-side flexibility.

---

## Overall Conclusions

### Integration of Parts (i) and (ii)

The dual formulation and demand/supply curves are intrinsically connected:

- **Shadow prices determine thresholds:** WTP and MOC derived directly from $|\pi_t|$ and tariffs
- **Dual variables guide interpretation:** High $|\pi_t|$ → high WTP → willing to pay more
- **Curves operationalize theory:** Convert theoretical shadow prices into actionable market strategies
- **KKT conditions explain behavior:** Import/export decisions follow predicted economic rationality

### Business Insights

**For Prosumers:**
1. Submit hourly demand/supply bids based on derived curves
2. Invest in battery storage when curtailment duals persistently positive
3. Shift flexible loads from high-$|\pi_t|$ to low-$|\pi_t|$ hours
4. Choose tariff structures minimizing no-trade zone width

**For System Operators:**
1. Use curve shapes to forecast consumer price response
2. Design tariffs encouraging beneficial flexibility
3. Procure grid services based on revealed WTP/MOC values

**For Market Designers:**
1. Ensure prices reflect system conditions to incentivize flexibility
2. Create flexibility products aligned with temporal value variation
3. Compensate flexibility provision based on opportunity costs (shadow prices)

### Value Proposition

This analysis demonstrates that demand-side flexibility has **quantifiable economic value**:

- Direct cost savings through optimized import/export
- Market revenue from exporting at favorable prices  
- System benefits through price-responsive demand
- Investment signals from dual variables

The absence of hard daily constraints (only penalty-based flexibility) maximizes this value while respecting consumer preferences.

---